<a href="https://colab.research.google.com/github/riyazahmadshaik/computationalmethods-5731/blob/master/In_class_exercise_08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The eighth in-class-exercise (20 points in total, 10/29/2020)**

The data for this exercise is from the dataset you created from assignment three. Please perform answer the following questions based on your data:

## (1) (10 points) Write a python program to extract the sentiment related terms from the corpus. You may use python package such as polyglot or external lexicon resources in the question. Rank the sentiment related terms by frequency.

In [ ]:
import pandas as pd
import nltk
import string
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
nltk.download('punkt')
from textblob import TextBlob

df = pd.read_csv("/content/SentimentReview.csv")

df['Content_Text'] = df['Content_Text'].apply(lambda x: " ".join(x for x in x.split() if x not in string.punctuation))
df['Content_Text'] = df['Content_Text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df['Content_Text'] = df['Content_Text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
print(df['Content_Text'].head())


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
0                                      amaz movi exist
1                               went second time watch
2               psycholog studi rather superhero flick
3    joaquin oscar joker best dark suspens thriller...
4                                         venic review
Name: Content_Text, dtype: object


In [ ]:
words=[]
for i in df['Content_Text']:
  words.append(nltk.word_tokenize(i))
print(words[:5])


[['amaz', 'movi', 'exist'], ['went', 'second', 'time', 'watch'], ['psycholog', 'studi', 'rather', 'superhero', 'flick'], ['joaquin', 'oscar', 'joker', 'best', 'dark', 'suspens', 'thriller', 'darker', 'dark', 'knight'], ['venic', 'review']]


In [ ]:
doc =[]
for i in range(len(df)-1):
  sentiment = (df['Sentiment'])[i]
  text = nltk.word_tokenize((df['Content_Text'])[i])
  doc.append((text,sentiment))

print(doc[:1])
import random
random.shuffle(doc)


[(['amaz', 'movi', 'exist'], 'Positive')]


In [ ]:
from nltk.probability import FreqDist
total_words=[]
for w in range(len(words)):
  total = nltk.FreqDist(words[w])
  total_words.append(list(total))
flat_list = [item for sublist in total_words for item in sublist]
flat_list[:20]

['amaz',
 'movi',
 'exist',
 'went',
 'second',
 'time',
 'watch',
 'psycholog',
 'studi',
 'rather',
 'superhero',
 'flick',
 'joaquin',
 'oscar',
 'joker',
 'best',
 'dark',
 'suspens',
 'thriller',
 'darker']

In [ ]:
def document_features(doc):
  doc_words=set(doc)
  features={}
  for w in flat_list:
    features['contains({})'.format(w)] = (w in doc_words)
  return features

In [ ]:
feature_sets = [(document_features(d),c) for (d,c) in doc]
train_data,test_data = feature_sets[:80],feature_sets[80:]
classifier = nltk.NaiveBayesClassifier.train(train_data)


In [ ]:
print(nltk.classify.accuracy(classifier,test_data))

0.5789473684210527


In [ ]:
classifier.show_most_informative_features(10)

Most Informative Features
          contains(film) = True           Neutra : Positi =      7.3 : 1.0
        contains(review) = True           Neutra : Positi =      4.4 : 1.0
        contains(endgam) = True           Neutra : Positi =      4.4 : 1.0
         contains(great) = True           Neutra : Positi =      4.4 : 1.0
       contains(everyon) = True           Neutra : Positi =      4.4 : 1.0
          contains(best) = True           Positi : Negati =      4.2 : 1.0
          contains(real) = True           Neutra : Negati =      2.9 : 1.0
    contains(masterpiec) = True           Positi : Negati =      2.9 : 1.0
           contains(act) = True           Neutra : Positi =      2.6 : 1.0
          contains(dark) = True           Positi : Negati =      2.0 : 1.0


## (2) (10 points) Compare the performance of the following tools in sentiment identification: TextBlob (https://textblob.readthedocs.io/en/dev/), VADER (https://github.com/cjhutto/vaderSentiment), TFIDF-based Support Vector Machine (SVM) (Split your data into training and testing data). Take your own annotation as the standard answers. 

Reference code: https://towardsdatascience.com/fine-grained-sentiment-analysis-in-python-part-1-2697bb111ed4

In [ ]:
#textblob
import textblob
from textblob import TextBlob
for i in df['Content_Text'][:20]:
  sent = TextBlob(i)
  print(sent.sentiment)

Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=0.2333333333333333, subjectivity=0.3666666666666667)
Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=0.1, subjectivity=0.65)
Sentiment(polarity=0.7, subjectivity=0.6000000000000001)
Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=0.2, subjectivity=0.30000000000000004)
Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=-0.5, subjectivity=0.2)
Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=1.0, subjectivity=0.3)
Sentiment(polarity=0.3, subjectivity=0.1)
Sentiment(polarity=0.5, subjectivity=0.5)
Sentiment(polarity=0.7, subjectivity=0.6000000000000001)
Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=0.0, subjectivity=0.0)


In [ ]:
#vader
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
analyzer = SentimentIntensityAnalyzer()
for i in df['Content_Text'][:20]:
  print(analyzer.polarity_scores(i))

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 0.496, 'pos': 0.504, 'compound': 0.7269}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 0.256, 'pos': 0.744, 'compound': 0.4404}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 0.667, 'pos': 0.333, 'compound': 0.128}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 1.0, 'neu': 0.0, 'pos': 0.0, 'compound': -0.5994}
{'neg': 0.0, 'neu': 0.4, 'pos': 0.6, 'compound': 0.128}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 0.588, 'pos': 0.412, 'compound': 0.6369}
{'neg': 0.442, 'neu': 0.209, 'pos': 0.349, 'compound': -0.1139}
{'neg': 0.0, 'neu': 0.312, 'pos': 0.688, 'compound': 0.296}
{'neg': 0.0, 'neu':

In [ ]:
#svm
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(
                    ngram_range = (1,3),
                    sublinear_tf = True,
                    max_features = 40000)
X_train=df[:70]
X_test=df[70:]
train_tv = tv.fit_transform(X_train['Content_Text'])
test_tv = tv.transform(X_test['Content_Text'])
from sklearn import svm
classifier_svm = svm.SVC(kernel='linear')
classifier_svm.fit(train_tv,X_train['Sentiment'])
predict = classifier_svm.predict(test_tv)

import sklearn
print(sklearn.metrics.accuracy_score(X_test['Sentiment'],predict))



0.5666666666666667


Vader: Used Vader for first 20 sentences. We can see from the result all the sentences all the neutral. We can see positive value is (0.504). So we believe vader would be accurate than textblob.

SVM:  We can see that the precision value is more for positive values and f-score of 0.5666666666666667. So, I can say positive values are accurate. So, I can say it is also accurate for upto postive values.

The number of positive sentences are 233, number of negative sentences are 366 and neutral are 210. That means, we can say that number of positives are more. 
In conclusion, I can say that vadar is better than SVM and Textblob. vadar is better when compared to SVM.